In [1]:
import pandas as pd
import numpy as np
import os
from osgeo import gdal
import re
# from pythermalcomfort.models import utci
from UTCI小数修改 import utci

In [2]:
weather_data = pd.read_csv(r'F:/TAMU_projects/1 Campus Digital Twin\Data\Dataset/Meteorological data/0 final for simulation/20220715.csv')
weather_data.head()

,Year,Month,Day,Hour,Minute,TEMP (C),RH (%),Wind Speed,Wind Direction,Rain,GHI (w/m2),DNI (w/m2),DHI (w/m2),Solar Zenith Angle
0,2022,7,15,0,0,26.67,71.35,2.570480,130.0,0,0,0,0,127.32
1,2022,7,15,1,0,26.11,76.79,2.056384,130.0,0,0,0,0,127.42
2,2022,7,15,2,0,25.56,79.10,1.799336,130.0,0,0,0,0,123.70
3,2022,7,15,3,0,25.56,81.55,1.542288,130.0,0,0,0,0,116.85
4,2022,7,15,4,0,25.56,81.55,2.056384,110.0,0,0,0,0,107.75


In [8]:
# average
tmrt_folder = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\Tmrt results\20220708'
output_folder = r'E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\UTCI results\20220708'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

tmrt_files = sorted([f for f in os.listdir(tmrt_folder) if f.endswith('.tif')])
tmrt_files

['merged_Tmrt_2022_189_average.tif']

In [9]:
for tmrt_file in tmrt_files:

    air_temperature = weather_data['TEMP (C)'].mean()
    wind_speed = weather_data['Wind Speed'].mean()
    relative_humidity = weather_data['RH (%)'].mean()

    tmrt_path = os.path.join(tmrt_folder, tmrt_file)
    ds = gdal.Open(tmrt_path)
    if ds is None:
        print(f"Error: Unable to open Tmrt file {tmrt_path}")
        continue
    tmrt = ds.GetRasterBand(1).ReadAsArray()
    gt = ds.GetGeoTransform()
    proj = ds.GetProjection()

    tmrt = np.where(tmrt == 0, np.nan, tmrt)

    utci_values = utci(tdb=air_temperature, tr=tmrt, v=wind_speed, rh=relative_humidity)

    utci_values = np.where(np.isnan(tmrt), 0, utci_values)

    output_file = os.path.join(output_folder, f'utci_2022_189_average.tif')
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
    if out_ds is None:
        print(f"Error: Unable to create output file {output_file}")
        continue
    out_ds.SetGeoTransform(gt)
    out_ds.SetProjection(proj)
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(utci_values)
    out_band.SetNoDataValue(0)  
    out_ds.FlushCache()
    out_ds = None

    print(f'UTCI计算完成并保存为 {output_file}')

UTCI计算完成并保存为 E:\TAMU study\Projects\1 Campus Digital Twin\1 Microclimate\Dataset\UTCI results\20220708\utci_2022_189_average.tif


In [3]:
# 24h
tmrt_folder = r'F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/Tmrt results/20220715/20220715_merged'
output_folder = r'F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

tmrt_files = sorted([f for f in os.listdir(tmrt_folder) if f.endswith('.tif')])

hour_pattern = re.compile(r'_(\d{4})[DN]\.tif')

In [5]:
for tmrt_file in tmrt_files:
    match = hour_pattern.search(tmrt_file)
    if not match:
        print(f"Warning: Skipping file {tmrt_file}, unable to extract hour information")
        continue
    hour_str = match.group(1)

    hour_int = int(hour_str[:2]) 
    weather_row = weather_data[weather_data['Hour'] == hour_int]
    if weather_row.empty:
        print(f"Warning: No weather data found for hour {hour_int}, skipping file {tmrt_file}")
        continue
    weather_row = weather_row.iloc[0]
    air_temperature = weather_row['TEMP (C)']
    wind_speed = weather_row['Wind Speed']
    relative_humidity = weather_row['RH (%)']

    tmrt_path = os.path.join(tmrt_folder, tmrt_file)
    ds = gdal.Open(tmrt_path)
    if ds is None:
        print(f"Error: Unable to open Tmrt file {tmrt_path}")
        continue
    tmrt = ds.GetRasterBand(1).ReadAsArray()
    gt = ds.GetGeoTransform()
    proj = ds.GetProjection()

    tmrt = np.where(tmrt == 0, np.nan, tmrt)

    utci_values = utci(tdb=air_temperature, tr=tmrt, v=wind_speed, rh=relative_humidity)

    utci_values = np.where(np.isnan(tmrt), 0, utci_values)

    output_file = os.path.join(output_folder, f'utci_2022_196_{hour_str}.tif')
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Float32)
    if out_ds is None:
        print(f"Error: Unable to create output file {output_file}")
        continue
    out_ds.SetGeoTransform(gt)
    out_ds.SetProjection(proj)
    out_band = out_ds.GetRasterBand(1)
    out_band.WriteArray(utci_values)
    out_band.SetNoDataValue(0)
    out_ds.FlushCache()
    out_ds = None

    print(f'UTCI计算完成并保存为 {output_file}')

UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0000.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0100.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0200.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0300.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0400.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0500.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0600.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_1209/20220715\utci_2022_196_0700.tif
UTCI计算完成并保存为 F:/TAMU_projects/1 Campus Digital Twin/Data/Dataset/UTCI results_12